In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style()

Чтобы поймать Петю за руку, как дешёвку, будем делать скользящее окно по парам значений. Утверждается, что в честном случае распределение пар значений будем равномерным. Чтобы отлавливать случаи, когда это не так, будем пользоваться хи-квадратом.

In [2]:
def slices_from_sequence(seq, size=2):
    result = []
    for i in range(len(seq) - size + 1):
        result.append(tuple(seq[i:i + size]))

    return result

In [3]:
def bin_counts(bins, seq):
    bins = {item : 0 for item in bins}
    for item in seq:
        if item not in bins.keys():
            raise Exception('unexpected item in sequence: %s' % str(item))
        bins[item] += 1
    return list(bins.values())

Выглядит справедливо. Теперь сделаем генераторы последовательностей по стратегиям Пети:

In [4]:
def genA(n: int = 100) -> np.ndarray:
    return np.random.randint(0, 2, size=n)

def genB(n: int = 100) -> np.ndarray:
    return np.array([1, 0] * (2 * n))

def genC(n: int = 100) -> np.ndarray:
    return np.array([1, 0] * n + [0, 1] * n)

def genD(n: int = 100) -> np.ndarray:
    head = [1, 1, 0, 0] * (n // 2)
    tail = [] if n % 2 == 0 else [1, 1]
    return np.array(head + tail)

def genE(n: int = 100) -> np.ndarray:
    head = [1, 1, 1, 0, 0, 0] * (n // 3)
    tail = [] if n % 3 == 0 else [1, 1] if n % 3 == 1 else [1, 1, 1, 0]
    return np.array(head + tail)

gens = {
    'A' : genA,
    'B' : genB,
    'C' : genC,
    'D' : genD,
    'E' : genE
}

Если мы сделаем окошко шириной 2, оно должно сработать во всех случаях, кроме D, на который мы напишем дополнительный тест, а пока что проверим на остальных:

In [5]:
from scipy.stats import chisquare

def test_2_slide(seq, bound: float = .05) -> bool:
    slices = slices_from_sequence(seq)
    counts = bin_counts((
        (0, 0),
        (0, 1),
        (1, 0),
        (1, 1),
    ), slices)
    _, pvalue = chisquare(counts)
    return pvalue > .05

In [6]:
REPEATS = 10_000

for name, generator in gens.items():
    rejected = 0
    for _ in range(REPEATS):
        seq = generator()
        if not test_2_slide(seq):
            rejected += 1
    
    print('Generator %s, %.2f rejected by 2-slide' % (name, rejected / REPEATS))

Generator A, 0.07 rejected by 2-slide
Generator B, 1.00 rejected by 2-slide
Generator C, 1.00 rejected by 2-slide
Generator D, 0.00 rejected by 2-slide
Generator E, 1.00 rejected by 2-slide


Как можно видеть, на пункте D наш подход даёт сбой, потому что для скользящего окна длины 2 он производит практически идеальные результаты. Сделаем второй тест, который будет делать скользящее окно длины 3, и совместим эти тесты:

In [7]:
def test_3_slide(seq, bound: float = .05) -> bool:
    slices = slices_from_sequence(seq, size=3)
    counts = bin_counts((
        (0, 0, 0),
        (0, 0, 1),
        (0, 1, 0),
        (0, 1, 1),
        (1, 0, 0),
        (1, 0, 1),
        (1, 1, 0),
        (1, 1, 1),
    ), slices)
    _, pvalue = chisquare(counts)
    return pvalue > .05

In [8]:
for name, generator in gens.items():
    rejected = 0
    for _ in range(REPEATS):
        seq = generator()
        if not test_3_slide(seq):
            rejected += 1
    
    print('Generator %s, %.2f rejected by 3-slide' % (name, rejected / REPEATS))

Generator A, 0.10 rejected by 3-slide
Generator B, 1.00 rejected by 3-slide
Generator C, 1.00 rejected by 3-slide
Generator D, 1.00 rejected by 3-slide
Generator E, 1.00 rejected by 3-slide


А, стоп, нам нормально и просто 3-slide сделать, я почему-то думал, что его E пройдет, а теперь понял, почему не пройдёт)

Надо теперь посчитать, начиная с каких n тест начинает работать:

In [9]:
for name, generator in gens.items():
    if name == 'A':
        continue # Strange to test this one
    
    n = 1
    while True:
        rejected = 0
        for _ in range(REPEATS):
            seq = generator(n=n)
            if not test_3_slide(seq):
                rejected += 1
        if rejected == REPEATS:
            print('n=%d is enough for generator %s' % (n, name))
            break
        n += 1

n=2 is enough for generator B
n=3 is enough for generator C


/home/covariance/.local/lib/python3.8/site-packages/scipy/stats/stats.py:6707: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp


n=1 is enough for generator D
n=1 is enough for generator E


Получается, для алгоритма достаточно $n=3$. Даже подозрительно хорошо. А, это понятно, у нас некоторые методы получают слишком мало повторений. Давайте попробуем все $n$ от $1$ до $10$.

In [10]:
for name, generator in gens.items():
    if name == 'A':
        continue # Strange to test this one
    rejects = []
    for n in range(1, 11):
        rejected = 0
        for _ in range(REPEATS):
            seq = generator(n=n)
            if not test_3_slide(seq):
                rejected += 1
        rejects.append(rejected / REPEATS)
    print('Generator %s: %s' % (name, str(rejects)))

Generator B: [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Generator C: [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Generator D: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
Generator E: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Как можно заметить, $n=10$ нам не хватает только для теста E, и это понятно, потому что там не встречаются только две перестановки.

In [11]:
rejects = []
for n in range(1, 25):
    rejected = 0
    for _ in range(REPEATS):
        seq = genE(n=n)
        if not test_3_slide(seq):
            rejected += 1
    rejects.append(rejected / REPEATS)
print('Generator E: %s' % str(rejects))

Generator E: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]


Теперь нам хватает $n=23$. В целом можно было бы попробовать комбинацию двух тестов, вдруг сойдётся при меньшем $n$:

In [12]:
for name, generator in gens.items():
    rejected = 0
    for _ in range(REPEATS):
        seq = generator()
        if (not test_3_slide(seq)) or (not test_2_slide(seq)):
            rejected += 1
    
    print('Generator %s, %.2f rejected by 2-3-slide' % (name, rejected / REPEATS))

Generator A, 0.11 rejected by 2-3-slide
Generator B, 1.00 rejected by 2-3-slide
Generator C, 1.00 rejected by 2-3-slide
Generator D, 1.00 rejected by 2-3-slide
Generator E, 1.00 rejected by 2-3-slide


От него нам нужно, чтобы мы довольно редко откидывали генератор А, поэтому добавим на отсечения внутри самих тестов поправку Бонферрони:

In [13]:
for name, generator in gens.items():
    rejected = 0
    for _ in range(REPEATS):
        seq = generator()
        if (not test_3_slide(seq, bound=0.025)) or (not test_2_slide(seq, bound=0.025)):
            rejected += 1
    
    print('Generator %s, %.2f rejected by 2-3-slide' % (name, rejected / REPEATS))

Generator A, 0.11 rejected by 2-3-slide
Generator B, 1.00 rejected by 2-3-slide
Generator C, 1.00 rejected by 2-3-slide
Generator D, 1.00 rejected by 2-3-slide
Generator E, 1.00 rejected by 2-3-slide


А теперь найдём достаточное $n$ для того, чтобы этот тест работал:

In [15]:
for name, generator in gens.items():
    if name == 'A':
        continue # Strange to test this one
    rejects = []
    for n in range(1, 21):
        rejected = 0
        for _ in range(REPEATS):
            seq = generator(n=n)
            if (not test_3_slide(seq, bound=0.025)) or (not test_2_slide(seq, bound=0.025)):
                rejected += 1
        rejects.append(rejected / REPEATS)
    print('Generator %s: %s' % (name, str(rejects)))

Generator B: [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Generator C: [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Generator D: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Generator E: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Ладно, улучшение не получилось.